In [1]:
!pip install spotipy --quiet

In [1]:
import sys
import spotipy
import spotipy.util as util
import argparse
import pprint
import os
import subprocess
import json
import pandas as pd
import time

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials


client_credentials_manager = SpotifyClientCredentials(client_id='<your id here>', 
                                                      client_secret='<your client secret here>')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
'''User clientid:"redacted"
User clientsecret: "redacted" '''

'User clientid:"redacted"\nUser clientsecret: "redacted" '

In [7]:
#from https://github.com/SylCard/Spotify-Emotions-Project/blob/master/getPlaylistData.py
#client_credentials_manager = SpotifyClientCredentials(client_id='<your id here>',
                                                      #client_secret='<your secret here>')


def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print (" %d %s %s" % (i, track['artists'][0]['name'],track['name']))

def get_track_features(track_id,sp):
    if track_id is None:
        return None
    else:
        features = sp.audio_features([track_id])
    return features

def get_features(tracks,sp):
    tracks_with_features=[]

    for track in tracks:
        features = get_track_features(track['id'],sp)
        print (track['name'])
        if not features:
            print("passing track %s" % track['name'])
            pass
        else:
            f = features[0]
            tracks_with_features.append(dict(
                                            name=track['name'],
                                            artist=track['artist'],
                                            id=track['id'],
                                            danceability=f['danceability'],
                                            energy=f['energy'],
                                            loudness=f['loudness'],
                                            speechiness=f['speechiness'],
                                            acousticness=f['acousticness'],
                                            tempo=f['tempo'],
                                            liveness=f['liveness'],
                                            valence=f['valence'],
                                            mode=f['mode']
                                            ))

        # time.sleep(0.1)

    # print(tracks_with_features[0])
    return tracks_with_features

def get_tracks_from_playlists(username, sp):
    playlists = sp.user_playlists(username)
    trackList = []
    for playlist in playlists['items']:
        if playlist['owner']['id'] == username:
            print (playlist['name'],' no. of tracks: ',playlist['tracks']['total'])
            results = sp.user_playlist(username, playlist['id'],fields="tracks,next")
            tracks = results['tracks']
            for i, item in enumerate(tracks['items']):
                track = item['track']
                trackList.append(dict(name=track['name'], id=track['id'], artist=track['artists'][0]['name']))

    # print(trackList[0])
    return trackList

def write_to_csv(track_features):
    df = pd.DataFrame(track_features)
    df.drop_duplicates(subset=['name','artist'])
    print ('Total tracks in data set', len(df))
    df.to_csv('SongsDataset.csv',index=False)

def main(username):
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    print ("Getting user tracks from playlists")
    tracks = get_tracks_from_playlists(username, sp)
    print ("Getting track audio features")
    tracks_with_features = get_features(tracks,sp)
    print ("Storing into csv")
    write_to_csv(tracks_with_features)


if __name__ == '__main__':
    print ('Starting...')
    parser = argparse.ArgumentParser(description='this sript will grab user playlists')
    parser.add_argument('--username', help='username')
    args = parser.parse_args()
    main(args.username)

Starting...


usage: ipykernel_launcher.py [-h] [--username USERNAME]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/jpate/Library/Jupyter/runtime/kernel-f625478f-3db1-44db-a178-cd0bcba79a28.json


SystemExit: 2

In [9]:
main('gmarie.snowhite')

HTTP Error for GET to https://api.spotify.com/v1/users/snowhite.gmarie/playlists with Params: {'limit': 50, 'offset': 0} returned 404 due to Not found.


Getting user tracks from playlists


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/users/snowhite.gmarie/playlists?limit=50&offset=0:
 Not found., reason: None